In [1]:
# Run this cell if using colab
%cd /content
!git clone https://github.com/Ziinc/nb-15-demo
%cd nb-15-demo
!pwd

/content
Cloning into 'nb-15-demo'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 20 (delta 2), reused 5 (delta 1), pack-reused 11
Unpacking objects: 100% (20/20), done.
/content/nb-15-demo
/content/nb-15-demo


In [2]:
!ls
!chmod u+x ./unpack
!./unpack
!ls

analysis.ipynb	data.partab  data.partad  unpack
data.partaa	data.partac  README.md
./data/NUSW-NB15_features.csv
./data/UNSW-NB15_1.csv
./data/UNSW-NB15_2.csv
./data/UNSW-NB15_3.csv
./data/UNSW-NB15_4.csv
./data/UNSW-NB15_LIST_EVENTS.csv
./data/UNSW_NB15_testing-set.csv
./data/UNSW_NB15_training-set.csv
analysis.ipynb	data.partaa  data.partac  data.tar.gz  unpack
data		data.partab  data.partad  README.md


# Data Cleaning

You can read about the features in the `./data/NUSW-NB15_features.csv` file

In [0]:
import pandas as pd

headers_df = pd.read_csv("data/NUSW-NB15_features.csv", encoding= 'unicode_escape')
headers_df.head()
column_names = headers_df["Name"].tolist()

In [6]:
column_names

['srcip',
 'sport',
 'dstip',
 'dsport',
 'proto',
 'state',
 'dur',
 'sbytes',
 'dbytes',
 'sttl',
 'dttl',
 'sloss',
 'dloss',
 'service',
 'Sload',
 'Dload',
 'Spkts',
 'Dpkts',
 'swin',
 'dwin',
 'stcpb',
 'dtcpb',
 'smeansz',
 'dmeansz',
 'trans_depth',
 'res_bdy_len',
 'Sjit',
 'Djit',
 'Stime',
 'Ltime',
 'Sintpkt',
 'Dintpkt',
 'tcprtt',
 'synack',
 'ackdat',
 'is_sm_ips_ports',
 'ct_state_ttl',
 'ct_flw_http_mthd',
 'is_ftp_login',
 'ct_ftp_cmd',
 'ct_srv_src',
 'ct_srv_dst',
 'ct_dst_ltm',
 'ct_src_ ltm',
 'ct_src_dport_ltm',
 'ct_dst_sport_ltm',
 'ct_dst_src_ltm',
 'attack_cat',
 'Label']

In [8]:

data_files = [
"data/UNSW-NB15_1.csv",
"data/UNSW-NB15_2.csv",
"data/UNSW-NB15_3.csv",
"data/UNSW-NB15_4.csv"
]
df = pd.concat((pd.read_csv(f, names=column_names) for f in data_files))

df.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DtypeWarning: Columns (1,3,47) have mixed types. Specify dtype option on import or set low_memory=False.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DtypeWarning: Columns (3,39,47) have mixed types. Specify dtype option on import or set low_memory=False.
  


,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,service,Sload,Dload,Spkts,Dpkts,swin,dwin,stcpb,dtcpb,smeansz,dmeansz,trans_depth,res_bdy_len,Sjit,Djit,Stime,Ltime,Sintpkt,Dintpkt,tcprtt,synack,ackdat,is_sm_ips_ports,ct_state_ttl,ct_flw_http_mthd,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,Label
0,59.166.0.0,1390,149.171.126.6,53,udp,CON,0.001055,132,164,31,29,0,0,dns,500473.93750,621800.93750,2,2,0,0,0,0,66,82,0,0,0.00000,0.000000,1421927414,1421927414,0.017,0.013000,0.0,0.0,0.0,0,0,0.0,0.0,0,3,7,1,3,1,1,1,NaN,0
1,59.166.0.0,33661,149.171.126.9,1024,udp,CON,0.036133,528,304,31,29,0,0,-,87676.08594,50480.17188,4,4,0,0,0,0,132,76,0,0,9.89101,10.682733,1421927414,1421927414,7.005,7.564333,0.0,0.0,0.0,0,0,0.0,0.0,0,2,4,2,3,1,1,2,NaN,0
2,59.166.0.6,1464,149.171.126.7,53,udp,CON,0.001119,146,178,31,29,0,0,dns,521894.53130,636282.37500,2,2,0,0,0,0,73,89,0,0,0.00000,0.000000,1421927414,1421927414,0.017,0.013000,0.0,0.0,0.0,0,0,0.0,0.0,0,12,8,1,2,2,1,1,NaN,0
3,59.166.0.5,3593,149.171.126.5,53,udp,CON,0.001209,132,164,31,29,0,0,dns,436724.56250,542597.18750,2,2,0,0,0,0,66,82,0,0,0.00000,0.000000,1421927414,1421927414,0.043,0.014000,0.0,0.0,0.0,0,0,0.0,0.0,0,6,9,1,1,1,1,1,NaN,0
4,59.166.0.3,49664,149.171.126.0,53,udp,CON,0.001169,146,178,31,29,0,0,dns,499572.25000,609067.56250,2,2,0,0,0,0,73,89,0,0,0.00000,0.000000,1421927414,1421927414,0.005,0.003000,0.0,0.0,0.0,0,0,0.0,0.0,0,7,9,1,1,1,1,1,NaN,0


In [9]:
df.describe

<bound method NDFrame.describe of                srcip  sport           dstip  ... ct_dst_src_ltm attack_cat Label
0         59.166.0.0   1390   149.171.126.6  ...              1        NaN     0
1         59.166.0.0  33661   149.171.126.9  ...              2        NaN     0
2         59.166.0.6   1464   149.171.126.7  ...              1        NaN     0
3         59.166.0.5   3593   149.171.126.5  ...              1        NaN     0
4         59.166.0.3  49664   149.171.126.0  ...              1        NaN     0
...              ...    ...             ...  ...            ...        ...   ...
440039    59.166.0.5  33094   149.171.126.7  ...              3        NaN     0
440040    59.166.0.7  20848   149.171.126.4  ...              2        NaN     0
440041    59.166.0.3  21511   149.171.126.9  ...              2        NaN     0
440042    59.166.0.9  35433   149.171.126.0  ...              2        NaN     0
440043  175.45.176.0  17293  149.171.126.17  ...              2   Exploits 

### Converting IPs

## Creating our Training, Validation , and Testing Sets
This step is essential to ensure that the model does not overfit to data that has been already seen.

Some [definitions](https://towardsdatascience.com/train-validation-and-test-sets-72cb40cba9e7):
- Training Dataset: For fitting the model
- Validation Dataset: For tuning hyperparameters
- Test Dataset: For evaluating final model

In [0]:
import numpy as np
from sklearn.model_selection import train_test_split

# We remove the labels from our dataset
# Note that our df variable still holds the original dataframe from the CSVs
X = df.drop(columns=["Label", "attack_cat"])
labels = df["Label"].values
cat = df["attack_cat"].values

For demonstrative purposes, we are only concerned with **identifying a possible attack**, and not concerned with categorizing the type of attack it is.

As such, we shall make use of the boolean labelling provided in the dataset.

In [0]:
# We split our dataset into training, test, and validation subsets.
# X_train and y_train is reassigned by the 2nd split.
# We also add in a different random state
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.25, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=33)


## Creating a Naive Model
To have a rough guiding basis of our model, we will compare further models against our Naive Model. This model will be a simple CART model.


In [12]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)

ValueError: ignored

In [0]:
tree.plot_tree(clf) 